In [76]:
import tensorflow as tf
import numpy as np

n_features = 28 * 28

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, n_features) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, n_features) / 255.
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [81]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name='kernel')
        b = tf.Variable(tf.zeros([n_neurons], name='bias'))
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z


n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

tf.reset_default_graph()

X = tf.placeholder(tf.float32,shape=(None,n_features), name='y')
y = tf.placeholder(tf.int32,shape=(None), name='y')

with tf.name_scope('dnn'):
#    hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
    hidden1 = neuron_layer(X, n_hidden1, name='hidden1', activation=tf.nn.relu)
#    hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name='hidden2', activation=tf.nn.relu)
#    logits = tf.layers.dense(hidden2, n_outputs, name='outputs')
    logits = neuron_layer(hidden2, n_outputs, name='outputs')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='mean')
    
learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
file_writer.close()
# tensorboard --logdir tf_logs/
# localhost:6006

In [83]:
n_epochs = 30
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_yepochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print('Epoch accuracy', epoch, 'Train', acc_train, 'Test', acc_test)
    saver.save(sess, './my_model_final.ckpt')

Epoch accuracy 0 Train 0.91253334 Test 0.9169
Epoch accuracy 1 Train 0.9323 Test 0.9332
Epoch accuracy 2 Train 0.94015 Test 0.9405
Epoch accuracy 3 Train 0.94918334 Test 0.9467
Epoch accuracy 4 Train 0.9546 Test 0.9505
Epoch accuracy 5 Train 0.95865 Test 0.9555
Epoch accuracy 6 Train 0.96293336 Test 0.9575
Epoch accuracy 7 Train 0.9659333 Test 0.9594
Epoch accuracy 8 Train 0.9687167 Test 0.9621
Epoch accuracy 9 Train 0.97061664 Test 0.9639
Epoch accuracy 10 Train 0.97263336 Test 0.9662
Epoch accuracy 11 Train 0.97471666 Test 0.9673
Epoch accuracy 12 Train 0.97651666 Test 0.9685
Epoch accuracy 13 Train 0.9777833 Test 0.9688
Epoch accuracy 14 Train 0.97713333 Test 0.9661
Epoch accuracy 15 Train 0.98005 Test 0.9699
Epoch accuracy 16 Train 0.9817333 Test 0.9709
Epoch accuracy 17 Train 0.9831 Test 0.9714
Epoch accuracy 18 Train 0.9834667 Test 0.9713
Epoch accuracy 19 Train 0.98255 Test 0.972
Epoch accuracy 20 Train 0.98501664 Test 0.972
Epoch accuracy 21 Train 0.9860333 Test 0.9733
Epoch ac

In [84]:
with tf.Session() as sess:
    saver.restore(sess, './my_model_final.ckpt')
    Z = logits.eval(feed_dict={X: X_test[:20]})

y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [85]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
